# dbitseq

In [ ]:
import anndata as ad
import numpy as np
import scanpy as sc
import pandas as pd


import seaborn as sns
from collections import Counter
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import sys
sys.path.append('..')
import matplotlib.pyplot as plt
from sklearn.metrics import adjusted_rand_score

from src.interface import UnitedNet
from src.configs import *
from scipy.stats import spearmanr, pearsonr

In [ ]:
def split_data(test_batch):
    adata_rna_train = adata_rna_all[adata_rna_all.obs['batch'] != test_batch]
    adata_morph_train = adata_morph_all[adata_morph_all.obs['batch'] != test_batch]
    adata_mrna_niche_train = adata_mrna_niche_all[adata_mrna_niche_all.obs['batch'] != test_batch]

    adata_rna_test = adata_rna_all[adata_rna_all.obs['batch'] == test_batch]
    adata_morph_test = adata_morph_all[adata_morph_all.obs['batch'] == test_batch]
    adata_mrna_niche_test = adata_mrna_niche_all[adata_mrna_niche_all.obs['batch'] == test_batch]

    return [adata_rna_train, adata_morph_train, adata_mrna_niche_train], [adata_rna_test, adata_morph_test,
                                                                          adata_mrna_niche_test]


In [ ]:
from sklearn import preprocessing
def change_label(adata,batch):
    adata.obs['batch'] = batch
    adata.obs['imagecol'] = adata.obs['array_col']
    adata.obs['imagerow'] = adata.obs['array_row']
    adata.obs['label'] = adata.obs['cell_type']
    return adata

In [ ]:
def pre_ps(adata_list,sc_pre = None):
    adata_list_all = [ad_x.copy() for ad_x in adata_list]
    scalars = []
    assert (adata_list_all[0].X>=0).all(), "poluted input"
    for idx, mod in enumerate(adata_list_all):
        t_x = mod.X
        if sc_pre != None:
            scaler = sc_pre[idx]
        else:
            scaler = preprocessing.StandardScaler().fit(t_x)
        t_x = scaler.transform(t_x)
        mod.X = t_x
        adata_list_all[idx] = mod
        scalars.append(scaler)

    return adata_list_all,scalars

# load dbitseq data

In [ ]:
technique = 'dbitseq'
data_path = f"../data/{technique}"
device = "cuda:0"

In [ ]:
adata_niche_rna_train = sc.read_h5ad(f'{data_path}/adata_niche_rna_train.h5ad')
adata_niche_rna_test = sc.read_h5ad(f'{data_path}/adata_niche_rna_test.h5ad')

adata_rna_train = sc.read_h5ad(f'{data_path}/adata_rna_train.h5ad')
adata_rna_test = sc.read_h5ad(f'{data_path}/adata_rna_test.h5ad')

adata_protein_train = sc.read_h5ad(f'{data_path}/adata_protein_train.h5ad')
adata_protein_test = sc.read_h5ad(f'{data_path}/adata_protein_test.h5ad')


In [ ]:
adata_rna_train = change_label(adata_rna_train,'train')
adata_protein_train=change_label(adata_protein_train,'train')
adata_niche_rna_train=change_label(adata_niche_rna_train,'train')

adata_rna_test = change_label(adata_rna_test,'test')
adata_protein_test = change_label(adata_protein_test,'test')
adata_niche_rna_test = change_label(adata_niche_rna_test,'test')

adatas_train = [adata_rna_train, adata_protein_train, adata_niche_rna_train]
adatas_test = [adata_rna_test, adata_protein_test, adata_niche_rna_test]

adatas_all = []
for ad_train, ad_test in zip(adatas_train,adatas_test):
    ad_all = ad_train.concatenate(ad_test,batch_key='sample')
    ad_all = change_label(ad_all,'test')
    adatas_all.append(ad_all)
adatas_all,_ = pre_ps(adatas_all)    
    

adatas_train,_ = pre_ps(adatas_train)   
adatas_test,_ = pre_ps(adatas_test)   


# Train and finetune UnitedNet

In [ ]:
train_model = True
if train_model:
    technique = 'dbitseq'
    data_path = f"../data/{technique}"
    root_save_path = f"../saved_results/dbitseq"
    model = UnitedNet(root_save_path, device=device, technique=dbitseq_config)
    model.train(adatas_train,verbose=True)
    model.finetune(adatas_all,verbose=True)
else:
    technique = 'dbitseq'
    data_path = f"../data/{technique}"
    root_save_path = f"../saved_results/dbitseq"
    model = UnitedNet(root_save_path, device=device, technique=dbitseq_config)



# Predict on all data

In [ ]:

model = UnitedNet(root_save_path, device=device, technique=dbitseq_config)
model.load_model(f"{root_save_path}/train_best.pt",device=torch.device(device))
model.model.device_in_use = device
predict_label = model.predict_label(adatas_all)

print(root_save_path,'ari:',adjusted_rand_score(adatas_all[0].obs['cell_type'],
                    predict_label))

In [ ]:
from src.data import create_dataloader
dataloader_test = create_dataloader(
    model.model,
    adatas_train,
    shuffle=False,
    batch_size=model.model.config["train_batch_size"],
)

In [ ]:
#smooth and plot results

coord=np.array((list(adatas_all[0].obs['array_row'].astype('int')),
                list(adatas_all[0].obs['array_col'].astype('int')))).T


united_clus=list(predict_label)

coord=np.array((list(adatas_all[0].obs['array_row'].astype('int')),
                list(adatas_all[0].obs['array_col'].astype('int')))).T

nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(coord)
distances,indices = nbrs.kneighbors(coord)

united_clus_new=[]
for indi,i in enumerate(united_clus):
  np.array(united_clus)[(indices[indi])]
  occurence_count=Counter(np.array(united_clus)[(indices[indi])])
  united_clus_new.append(occurence_count.most_common(1)[0][0])
    

cluster_pl = sns.color_palette('tab20',20)
color_list = [cluster_pl[5],
 cluster_pl[1],
 cluster_pl[2],
 cluster_pl[4],
 cluster_pl[11],
 cluster_pl[6],
 cluster_pl[3],
 cluster_pl[7],
 cluster_pl[8],
 cluster_pl[0]]    

In [ ]:
plt.figure(figsize=(6,5))
for idx,clus_id in enumerate(set(united_clus_new)):
    
    plt.scatter(adatas_all[0].obs['array_row'][(united_clus_new==clus_id)],
               adatas_all[0].obs['array_col'][(united_clus_new==clus_id)],
               color=color_list[idx],cmap='tab20')
plt.axis('off')
